In [1]:
import os

base_dir = "Results_ResNet50"

results = []

for folder_name in os.listdir(base_dir):
    # Verifica se a pasta contém o padrão "(X%)" no nome
    if '%)' not in folder_name:
        continue
    
    try:
        # Extrai o valor de X% (ex: 0.1, 5, 10.5, etc.)
        before_percent = folder_name.split('%)')[0]
        x_part = before_percent.split('(')[-1]
        x_str = x_part.replace('%', '').strip()
        x = float(x_str)
    except (IndexError, ValueError):
        print(f"Ignorando pasta com formato inválido: {folder_name}")
        continue
    
    folder_path = os.path.join(base_dir, folder_name)
    metrics = {
        'accuracy': [],
        'jaccard': [],
        'f1': [],
        'precision': [],
        'recall': []
    }
    
    for fold in range(1, 6):
        file_name = f"metrics_Yildirim_fold_{fold}.txt"
        file_path = os.path.join(folder_path, file_name)
        
        if not os.path.exists(file_path):
            print(f"Arquivo não encontrado: {file_path}")
            continue
        
        fold_metrics = {
            'accuracy': None,
            'jaccard': None,
            'f1': None,
            'precision': None,
            'recall': None
        }
        
        with open(file_path, 'r') as file:
            for line in file:
                line = line.strip()
                if line.startswith('Accuracy: '):
                    fold_metrics['accuracy'] = float(line.split()[1])
                elif line.startswith('Jaccard Score: '):
                    fold_metrics['jaccard'] = float(line.split()[2])
                elif line.startswith('F1 Score: '):
                    fold_metrics['f1'] = float(line.split()[2])
                elif line.startswith('Precision: '):
                    fold_metrics['precision'] = float(line.split()[1])
                elif line.startswith('Recall: '):
                    fold_metrics['recall'] = float(line.split()[1])
        
        if all(value is not None for value in fold_metrics.values()):
            for key in metrics:
                metrics[key].append(fold_metrics[key])
        else:
            print(f"Métricas incompletas em: {file_path}. Fold {fold} ignorado.")
    
    if all(len(lst) == 5 for lst in metrics.values()):
        avg_metrics = {
            'accuracy': sum(metrics['accuracy']) / 5,
            'jaccard': sum(metrics['jaccard']) / 5,
            'f1': sum(metrics['f1']) / 5,
            'precision': sum(metrics['precision']) / 5,
            'recall': sum(metrics['recall']) / 5
        }
        results.append((x, avg_metrics))
    else:
        print(f"Pasta {folder_name} ignorada. Métricas coletadas: {{accuracy: {len(metrics['accuracy'])}, jaccard: {len(metrics['jaccard'])}, ...}}")

results.sort(key=lambda item: item[0])

print("Resultados finais (X% | Acurácia | Jaccard | F1 | Precision | Recall):")
for x, avg in results:
    print(f"{x}%: "
          f"Acurácia={avg['accuracy']:.4f}, "
          f"Jaccard={avg['jaccard']:.4f}, "
          f"F1={avg['f1']:.4f}, "
          f"Precision={avg['precision']:.4f}, "
          f"Recall={avg['recall']:.4f}")

Resultados finais (X% | Acurácia | Jaccard | F1 | Precision | Recall):
0.1%: Acurácia=1.0000, Jaccard=1.0000, F1=1.0000, Precision=1.0000, Recall=1.0000
